In [1]:
!pip install langchain openai qdrant_client tiktoken

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddlenlp 2.6.1 requires protobuf==3.20.2, but you have protobuf 4.24.4 which is incompatible.
paddlepaddle 2.5.1 requires protobuf<=3.20.2,>=3.1.0; platform_system == "Windows", but you have protobuf 4.24.4 which is incompatible.
pymilvus 2.2.11 requires grpcio<=1.53.0,>=1.49.1, but you have grpcio 1.59.0 which is incompatible.


     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     --------------------- ------------------ 30.7/57.5 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 57.5/57.5 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/181.3 kB ? eta -:--:--
   ------------------------ --------------- 112.6/181.3 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 181.3/181.3 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB 7.9 MB/s eta 0:00:01
   ---------------------- ----------------- 0.6/1.1 MB 7.5 MB/s eta 0:00:01
   ---------------------- ----------------- 0.6/1.1 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/430.5 kB ? eta -:--:--
   ------------- 

In [12]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
import qdrant_client
import os

In [4]:
os.environ["QDRANT_HOST"] = "****"
os.environ["QDRANT_API_KEY"] = "*****"

client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [6]:
from qdrant_client.http import models

os.environ["QDRANT_COLLECTION_NAME"] = "my-collection"

vectors_config = qdrant_client.http.models.VectorParams(size=1536, distance=models.Distance.COSINE) 

In [7]:
client.create_collection(os.getenv("QDRANT_COLLECTION_NAME"), vectors_config)

True

create vector store

In [13]:
os.environ["OPENAI_API_KEY"] = "sk-*****"

embeddings = OpenAIEmbeddings()

vectorstore = Qdrant(
    client=client,
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
    embeddings=embeddings,
)

add documents

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

def get_chunks(text: str):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_over_lap=200,
    )

    chunks = text_splitter.split_text(text)
    return chunks

with open("story.txt") as f:
    raw_text = f.read()
    
texts = get_chunks(raw_text)

vectorstore.add_texts(texts)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI


qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
    )

In [ ]:
query = "How many friends are there and what are their names?"
response = qa.run(query)

print(response)